# Pydeck Earth Engine Introduction

This is an introduction to using [Pydeck](https://pydeck.gl) and [Deck.gl](https://deck.gl) with [Google Earth Engine](https://earthengine.google.com/) in Jupyter Notebooks.

If you wish to run this locally, you'll need to install some dependencies. Installing into a new Conda environment is recommended. To create and enter the environment, run:
```
conda create -n pydeck-ee -c conda-forge python jupyter notebook pydeck earthengine-api requests -y
source activate pydeck-ee
jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck
jupyter nbextension enable --sys-prefix --py pydeck
```
then open Jupyter Notebook with `jupyter notebook`.

Now in a Python Jupyter Notebook, let's first import required packages:

In [1]:
from pydeck_earthengine_layers import EarthEngineLayer
import pydeck as pdk
import requests
import ee

## Authentication

Using Earth Engine requires authentication. If you don't have a Google account approved for use with Earth Engine, you'll need to request access. For more information and to sign up, go to https://signup.earthengine.google.com/.

If you haven't used Earth Engine in Python before, you'll need to run the following authentication command. If you've previously authenticated in Python or the command line, you can skip the next line.

Note that this creates a prompt which waits for user input. If you don't see a prompt, you may need to authenticate on the command line with `earthengine authenticate` and then return here, skipping the Python authentication.

In [ ]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create Map

Next it's time to create a map. Here we create an `ee.Image` object

In [ ]:
# Initialize objects
ee_layers = []
view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=10, bearing=0, pitch=45)

In [4]:
# %%
# Add Earth Engine dataset
# Define a region of interest as a point.  Change the coordinates
# to get a classification of any place where there is imagery.
roi = ee.Geometry.Point(-122.3942, 37.7295)

# Load Landsat 5 input imagery.
landsat = ee.Image(ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA')
  # Filter to get only one year of images. \
  .filterDate('2011-01-01', '2011-12-31')
  # Filter to get only images under the region of interest. \
  .filterBounds(roi)
  # Sort by scene cloudiness, ascending. \
  .sort('CLOUD_COVER')
  # Get the first (least cloudy) scene. \
  .first())

# Compute cloud score.
cloudScore = ee.Algorithms.Landsat.simpleCloudScore(landsat).select('cloud')

# Mask the input for clouds.  Compute the min of the input mask to mask
# pixels where any band is masked.  Combine that with the cloud mask.
input = landsat.updateMask(landsat.mask().reduce('min').And(cloudScore.lte(50)))

# Use MODIS land cover, IGBP classification, for training.
modis = ee.Image('MODIS/051/MCD12Q1/2011_01_01') \
    .select('Land_Cover_Type_1')

# Sample the input imagery to get a FeatureCollection of training data.
training = input.addBands(modis).sample(**{
  'numPixels': 5000,
  'seed': 0
})

# Make a Random Forest classifier and train it.
classifier = ee.Classifier.randomForest(10) \
    .train(training, 'Land_Cover_Type_1')

# Classify the input imagery.
classified = input.classify(classifier)

# Get a confusion matrix representing resubstitution accuracy.
trainAccuracy = classifier.confusionMatrix()
print('Resubstitution error matrix: ', trainAccuracy)
print('Training overall accuracy: ', trainAccuracy.accuracy())

# Sample the input with a different random seed to get validation data.
validation = input.addBands(modis).sample(**{
  'numPixels': 5000,
  'seed': 1
  # Filter the result to get rid of any {} pixels.
}).filter(ee.Filter.neq('B1', {}))

# Classify the validation data.
validated = validation.classify(classifier)

# Get a confusion matrix representing expected accuracy.
testAccuracy = validated.errorMatrix('Land_Cover_Type_1', 'classification')
print('Validation error matrix: ', testAccuracy)
print('Validation overall accuracy: ', testAccuracy.accuracy())

# Define a palette for the IGBP classification.
igbpPalette = [
  'aec3d4', # water
  '152106', '225129', '369b47', '30eb5b', '387242', # forest
  '6a2325', 'c3aa69', 'b76031', 'd9903d', '91af40',  # shrub, grass
  '111149', # wetlands
  'cdb33b', # croplands
  'cc0013', # urban
  '33280d', # crop mosaic
  'd7cdcc', # snow and ice
  'f7e084', # barren
  '6f6f6f'  # tundra
]

# Display the input and the classification.
ee_layers.append(EarthEngineLayer(ee_object=input, vis_params={'bands':['B3','B2','B1'],'max':0.4}))
ee_layers.append(EarthEngineLayer(ee_object=classified, vis_params={'palette':igbpPalette,'min':0,'max':17}))





Then just pass these layers to a `pydeck.Deck` instance, and call `.show()` to create a map:

In [8]:
r = pdk.Deck(layers=ee_layers, initial_view_state=view_state)
r.show()

DeckGLWidget(custom_libraries=[{'libraryName': 'EarthEngineLayerLibrary', 'resourceUri': 'https://cdn.jsdelivr…